In [1]:
import os
%pwd

'/Users/deryliankhalfani/Documents/Self_Learn/Udemy/MLOps Bootcamp/End2EndMLOpsBootcamp_NLP/research'

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params_filepath = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_configuration(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,

        )

        return data_transformation_config

## Components

In [6]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-08-30 15:57:07,312: INFO: config: TensorFlow version 2.19.0 available. ]


In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
      input_encodings = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation=True)

      with self.tokenizer.as_target_tokenizer():
        target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation=True)

      return {
          'input_ids': input_encodings['input_ids'],
          'attention_mask': input_encodings['attention_mask'],
          'labels': target_encodings['input_ids']
      }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))

In [8]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_configuration()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-08-30 15:57:07,601: INFO: common: yaml file: config/config.yaml loaded successfully ]
[2025-08-30 15:57:07,605: INFO: common: yaml file: params.yaml loaded successfully ]
[2025-08-30 15:57:07,606: INFO: common: Created directory at: artifacts ]
[2025-08-30 15:57:07,607: INFO: common: Created directory at: artifacts/data_transformation ]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 144497.16 examples/s]
